In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, DensityMatrix, partial_trace
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, QuantumError
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

SIMULATION_PARAMS = {
    3:  [1e-5, 2e-5,1e-6,2e-7], 4:  [1e-6, 3e-6,1e-7,4e-8],5:  [3e-7, 1e-8, 4e-8, 7e-8],6:  [5e-9, 7e-9, 1e-8, 4e-8]
}
TSAC_NUM_ROUNDS = 6

h_planck = 6.62607015e-34; k_boltzmann = 1.380649e-23
f_qubit_GHz = 10.0; f_qubit_Hz = f_qubit_GHz * 1e9
energy_gap = h_planck * f_qubit_Hz

def temperature_to_population(temp_K, energy_gap_J):
    if temp_K == 0: return 1.0
    beta = 1.0 / (k_boltzmann * temp_K)
    return 1.0 / (1.0 + np.exp(-beta * energy_gap_J))

def population_to_temperature(pop_ground, energy_gap_J):
    pop_ground = np.clip(pop_ground, 0.0, 1.0)
    if pop_ground >= 1.0: return 0.0
    if pop_ground <= 0.5: return float('inf')
    ratio = (1.0 - pop_ground) / pop_ground
    return -energy_gap_J / (k_boltzmann * np.log(ratio))

GLOBAL_BASIS_GATES = ["cx", "sx", "rz"]
def create_noise_model(p_cx):
    p_sx = p_cx / 10.0; p_rz = p_cx / 10.0
    I = np.eye(2, dtype=np.complex128); X = np.array([[0, 1], [1, 0]], dtype=np.complex128)
    Z = np.array([[1, 0], [0, -1]], dtype=np.complex128); P0 = np.array([[1, 0], [0, 0]]); P1 = np.array([[0, 0], [0, 1]])
    sx_err = QuantumError(Kraus([np.sqrt(1 - p_sx) * I, np.sqrt(p_sx) * X]))
    rz_err = QuantumError(Kraus([np.sqrt(1 - p_rz) * I, np.sqrt(p_rz) * Z]))
    cx_err = QuantumError(Kraus([np.sqrt(1 - p_cx) * np.kron(I, I), np.sqrt(p_cx) * (np.kron(P0, I) + np.kron(P1, X))]))
    noise_model = NoiseModel(); noise_model.add_all_qubit_quantum_error(sx_err, 'sx')
    noise_model.add_all_qubit_quantum_error(rz_err, 'rz'); noise_model.add_all_qubit_quantum_error(cx_err, 'cx')
    return noise_model
def apply_global_depolarizing(rho, p):
    n = rho.num_qubits; dim = 2 ** n
    identity_part = np.eye(dim, dtype=np.complex128) / dim
    return DensityMatrix((1 - p) * rho.data + p * identity_part)
def run_tsac_simulation(n, p_cx, num_rounds, initial_temp_K):
    qc = QuantumCircuit(n)
    if n > 1:
        qc.x(n-1); [qc.mcx(list(range(i+1, n)), i) for i in range(n-2, -1, -1)]
        qc.mcx(list(range(n-1)), n-1); qc.x(n-1)
        [qc.mcx(list(range(i+1, n)), i) for i in range(n-1)]; qc.x(n-1)
    decomposed_U = transpile(qc, basis_gates=GLOBAL_BASIS_GATES, optimization_level=0)
    cnot_count = decomposed_U.count_ops().get('cx', 0)
    initial_pop = temperature_to_population(initial_temp_K, energy_gap)
    rho_single = DensityMatrix(np.diag([initial_pop, 1-initial_pop]))
    noise_model = create_noise_model(p_cx)
    simulator_phys = AerSimulator(method='density_matrix', noise_model=noise_model)
    current_rho_phys = rho_single.copy()
    for _ in range(n-1): current_rho_phys = current_rho_phys.tensor(rho_single)
    for _ in range(num_rounds):
        qc_run = QuantumCircuit(n); qc_run.set_density_matrix(current_rho_phys)
        qc_run.compose(decomposed_U, inplace=True); qc_run.save_state()
        current_rho_phys = DensityMatrix(simulator_phys.run(qc_run).result().data(0)['density_matrix'])
        traced = partial_trace(current_rho_phys, [n-1])
        current_rho_phys = rho_single.tensor(traced)
    final_pop_phys = partial_trace(current_rho_phys, list(range(1, n))).data[0,0].real
    T_sim = population_to_temperature(final_pop_phys, energy_gap)
    p_depolarizing = max(0.0, min(p_cx * cnot_count * 0.75 * (2**n)**2 / ((2**n)**2 - 1) , 1.0)) # Corrected p_depolarizing for Timekeeping Noise
    simulator_ideal = AerSimulator(method='density_matrix')
    current_rho_model = rho_single.copy()
    for _ in range(n-1): current_rho_model = current_rho_model.tensor(rho_single)
    for _ in range(num_rounds):
        qc_run = QuantumCircuit(n); qc_run.set_density_matrix(current_rho_model)
        qc_run.compose(decomposed_U, inplace=True); qc_run.save_state()
        rho_ideal_step = DensityMatrix(simulator_ideal.run(qc_run).result().data(0)['density_matrix'])
        current_rho_model = apply_global_depolarizing(rho_ideal_step, p_depolarizing)
        traced = partial_trace(current_rho_model, [n-1])
        current_rho_model = rho_single.tensor(traced)
    final_pop_model = partial_trace(current_rho_model, list(range(1, n))).data[0,0].real
    T_model = population_to_temperature(final_pop_model, energy_gap)
    return T_sim, T_model

try:
    from qubitcooling import DynamicCooling, MirrorProtocol
except ImportError:
    print("Could not import qubitcooling library. DC simulation will be skipped.")
    MirrorProtocol = None
def run_dc_mirror_simulation(n, p_cx, initial_temp_K):
    if not MirrorProtocol: return float('nan'), float('nan')
    unitary_obj = MirrorProtocol(n); cooling_obj = DynamicCooling(unitary_obj, False)
    cooling_circuit_raw = cooling_obj.getCircuit()
    decomposed_U = transpile(cooling_circuit_raw, basis_gates=GLOBAL_BASIS_GATES, optimization_level=0)
    cnot_count = decomposed_U.count_ops().get('cx', 0)
    initial_pop = temperature_to_population(initial_temp_K, energy_gap)
    rho_single = DensityMatrix(np.diag([initial_pop, 1-initial_pop]))
    rho_initial = rho_single.copy()
    for _ in range(n-1): rho_initial = rho_initial.tensor(rho_single)
    qc_run = QuantumCircuit(n); qc_run.set_density_matrix(rho_initial)
    qc_run.compose(decomposed_U, inplace=True); qc_run.save_state()
    noise_model = create_noise_model(p_cx)
    simulator_phys = AerSimulator(method='density_matrix', noise_model=noise_model)
    rho_final_phys = DensityMatrix(simulator_phys.run(qc_run).result().data(0)['density_matrix'])
    final_pop_phys = partial_trace(rho_final_phys, list(range(n - 1))).data[0,0].real
    T_sim = population_to_temperature(final_pop_phys, energy_gap)
    p_depolarizing = max(0.0, min(p_cx * cnot_count * 0.75 * (2**n)**2 / ((2**n)**2 - 1) , 1.0)) # Corrected p_depolarizing for Timekeeping Noise
    simulator_ideal = AerSimulator(method='density_matrix')
    rho_final_ideal = DensityMatrix(simulator_ideal.run(qc_run).result().data(0)['density_matrix'])
    rho_final_model = apply_global_depolarizing(rho_final_ideal, p_depolarizing)
    final_pop_model = partial_trace(rho_final_model, list(range(n - 1))).data[0,0].real
    T_model = population_to_temperature(final_pop_model, energy_gap)
    return T_sim, T_model







def plot_dc_mirror_only(results_data, output_dir, simulated_n_values):

    plt.rcParams.update({
        "font.family": "serif", "font.serif": ["Times New Roman"], "mathtext.fontset": "stix",
        "font.size": 9, "axes.labelsize": 9, "axes.titlesize": 9, "xtick.labelsize": 9,
        "ytick.labelsize": 9, "legend.fontsize": 9, "axes.linewidth": 0.5, "lines.linewidth": 1.0,
        "xtick.major.width": 0.5, "ytick.major.width": 0.5, "xtick.direction": "in",
        "ytick.direction": "in", "xtick.top": True, "ytick.right": True,
        "pdf.fonttype": 42, "ps.fonttype": 42
    })

    fig_width_inch = 3.375
    fig_height_inch = fig_width_inch * 0.85  # 调整高度比例，适应2行

    data_B_raw = [d for d in results_data if d['protocol'] == 'DC_Mirror']

    def filter_finite_data(raw_data):
        finite_data = [d for d in raw_data if np.isfinite(d['T_sim']) and np.isfinite(d['T_model'])]
        if not finite_data:
             return {'n': [], 'p': [], 'T_sim_K': np.array([]), 'T_model_K': np.array([]), 'T_rel_err_percent': np.array([])}
        
        t_sim_arr = np.array([d['T_sim'] for d in finite_data])
        t_model_arr = np.array([d['T_model'] for d in finite_data])
        t_diff_abs = t_sim_arr - t_model_arr
        t_rel_err = np.divide(np.abs(t_diff_abs), t_model_arr, 
                              out=np.full_like(t_model_arr, np.nan), 
                              where=t_model_arr!=0)
        
        t_rel_err_percent = t_rel_err * 100.0 
        
        return {
            'n': [d['n'] for d in finite_data], 'p': [d['p'] for d in finite_data],
            'T_sim_K': t_sim_arr,
            'T_model_K': t_model_arr,
            'T_rel_err_percent': t_rel_err_percent
        }

    data_B = filter_finite_data(data_B_raw)
    if not data_B['n']:
        print("Warning: No finite DC Mirror data to plot.")
        return

    data_B['T_sim_mK'] = data_B['T_sim_K'] * 1000
    data_B['T_model_mK'] = data_B['T_model_K'] * 1000

    all_temps_mK = np.concatenate([data_B['T_sim_mK'], data_B['T_model_mK']])
    finite_temps = all_temps_mK[np.isfinite(all_temps_mK)]
    vmin_temp, vmax_temp = (np.min(finite_temps), np.max(finite_temps)) if len(finite_temps) > 0 else (0,1)
    cmap_temp = 'viridis'

    all_rel_errs_percent = data_B['T_rel_err_percent']
    finite_rel_errs = all_rel_errs_percent[np.isfinite(all_rel_errs_percent) & (all_rel_errs_percent > 0)]
    vmin_rel, vmax_rel = (np.min(finite_rel_errs), np.max(finite_rel_errs)) if len(finite_rel_errs) > 0 else (0, 1.0)
    cmap_rel = 'cividis' 

    fig, axes = plt.subplots(2, 1, sharex=True, figsize=(fig_width_inch, fig_height_inch),
                             gridspec_kw={'hspace': 0.15, 'height_ratios': [1, 1]}) 
    ax0, ax1 = axes  

    s_size = 20 
    
    sc_temp0 = ax0.scatter(data_B['n'], data_B['p'], c=data_B['T_sim_mK'], 
                           cmap=cmap_temp, vmin=vmin_temp, vmax=vmax_temp, 
                           s=s_size, edgecolors='k', lw=0.4)
    
    sc_rel = ax1.scatter(data_B['n'], data_B['p'], c=data_B['T_rel_err_percent'], 
                         cmap=cmap_rel, vmin=vmin_rel, vmax=vmax_rel, 
                         s=s_size, edgecolors='k', lw=0.4)

    ax0.text(0.0, 1.035, '(a)', transform=ax0.transAxes, fontsize=9, weight='bold', va='baseline', ha='left')
    ax0.text(0.1, 1.02, 'Physical Simulation', transform=ax0.transAxes, fontsize=9, va='baseline', ha='left')

    ax1.text(0.0, 1.035, '(b)', transform=ax1.transAxes, fontsize=9, weight='bold', va='baseline', ha='left')
    ax1.text(0.1, 1.02, r'Relative Error $|\mathcal{T}_{\mathrm{sim}}\!-\!\mathcal{T}_{\mathrm{model}}|/\mathcal{T}_{\mathrm{model}}$', transform=ax1.transAxes, fontsize=9, va='baseline', ha='left')

    ax1.set_xlabel('Number of Qubits ($n$)') 
    ax1.set_xticks(simulated_n_values)

    for ax in axes:
        ax.grid(True, linestyle=':', alpha=0.7)
        ax.set_yscale('log')
        ax.minorticks_off()
        if data_B['p']:
            min_p = min(data_B['p']); max_p = max(data_B['p'])
            ax.set_ylim(min_p * 0.5, max_p * 2.0)

    cax_temp_pos = [0.82, 0.6, 0.03, 0.30] 
    cax_temp = fig.add_axes(cax_temp_pos)
    cb_temp = fig.colorbar(sc_temp0, cax=cax_temp)
    cb_temp.set_label(r'$\mathcal{T}$ (mK)', size=8) 
    cb_temp.ax.tick_params(labelsize=8)

    cax_rel_pos = [0.82, 0.188, 0.03, 0.30] 
    cax_rel = fig.add_axes(cax_rel_pos)
    cb_rel = fig.colorbar(sc_rel, cax=cax_rel, format='%.1f%%') 
    cb_rel.set_label('Error (%)', size=8) 
    cb_rel.ax.tick_params(labelsize=8)
    cb_rel.locator = plt.MaxNLocator(nbins=4)
    cb_rel.update_ticks()
    fig.text(0.07, 0.535, 'Error Prob. ($p$)', va='center', ha='center', rotation='vertical', fontsize=9)
    fig.subplots_adjust(left=0.20, right=0.78, top=0.92, bottom=0.15) 

    # --- 保存 ---
    file_name = "dc_mirror_compact_2panel_v1.pdf"
    full_path = os.path.join(output_dir, file_name)
    plt.savefig(full_path, dpi=300)
    print(f"Compact 2-panel figure saved to: {full_path}")
    plt.close(fig)

def plot_comprehensive_comparison(results_data, output_dir, simulated_n_values):

    plt.rcParams.update({
    "font.family": "serif", "font.serif": ["Times New Roman"], "mathtext.fontset": "stix",
    "font.size": 9, "axes.labelsize": 9, "axes.titlesize": 9, "xtick.labelsize": 9,
    "ytick.labelsize": 9, "legend.fontsize": 9, "axes.linewidth": 0.5, "lines.linewidth": 1.0,
    "xtick.major.width": 0.5, "ytick.major.width": 0.5, "xtick.direction": "in",
    "ytick.direction": "in", "xtick.top": True, "ytick.right": True,
    "pdf.fonttype": 42,
    "ps.fonttype": 42
    })
    fig_width_inch = 7.0
    fig_height_inch = fig_width_inch * (5.0 / 8.0)

    data_A_raw = [d for d in results_data if d['protocol'] == 'TSAC']
    data_B_raw = [d for d in results_data if d['protocol'] == 'DC_Mirror']
    
    def filter_finite_data(raw_data):
        finite_data = [d for d in raw_data if np.isfinite(d['T_sim']) and np.isfinite(d['T_model'])]
        if not finite_data:
             return {'n': [], 'p': [], 'T_sim_K': np.array([]), 'T_model_K': np.array([]), 'T_rel_err_percent': np.array([])} # !! 修改
        
        t_sim_arr = np.array([d['T_sim'] for d in finite_data])
        t_model_arr = np.array([d['T_model'] for d in finite_data])
        t_diff_abs = t_sim_arr - t_model_arr
        t_rel_err = np.divide(np.abs(t_diff_abs), t_model_arr, 
                              out=np.full_like(t_model_arr, np.nan), 
                              where=t_model_arr!=0)
        
        t_rel_err_percent = t_rel_err * 100.0 # !! 乘以 100 !!
        
        return {
            'n': [d['n'] for d in finite_data], 'p': [d['p'] for d in finite_data],
            'T_sim_K': t_sim_arr,
            'T_model_K': t_model_arr,
            'T_rel_err_percent': t_rel_err_percent # !! 返回百分比 !!
        }
        
    data_A = filter_finite_data(data_A_raw)
    data_B = filter_finite_data(data_B_raw)

    if not data_A['n'] and not data_B['n']:
        print("Warning: No finite data for either protocol to plot.")
        return

    # 转换为 mK
    if data_A['n']:
        data_A['T_sim_mK'] = data_A['T_sim_K'] * 1000
        data_A['T_model_mK'] = data_A['T_model_K'] * 1000
    if data_B['n']:
        data_B['T_sim_mK'] = data_B['T_sim_K'] * 1000
        data_B['T_model_mK'] = data_B['T_model_K'] * 1000

    all_temps_mK = np.concatenate([data_A.get('T_sim_mK', []), data_A.get('T_model_mK', []), data_B.get('T_sim_mK', []), data_B.get('T_model_mK', [])])
    finite_temps = all_temps_mK[np.isfinite(all_temps_mK)]; vmin_temp, vmax_temp = (np.min(finite_temps), np.max(finite_temps)) if len(finite_temps) > 0 else (0,1)
    cmap_temp = 'viridis'

    all_rel_errs_percent = np.concatenate([data_A.get('T_rel_err_percent', []), data_B.get('T_rel_err_percent', [])])
    finite_rel_errs = all_rel_errs_percent[np.isfinite(all_rel_errs_percent) & (all_rel_errs_percent > 0)]
    vmin_rel, vmax_rel = (np.min(finite_rel_errs), np.max(finite_rel_errs)) if len(finite_rel_errs) > 0 else (0, 1.0)
    cmap_rel = 'cividis' 

    fig = plt.figure(figsize=(fig_width_inch, fig_height_inch))
    gs = gridspec.GridSpec(2, 6, figure=fig,
                           width_ratios=[15, 15, 1, 3.0, 15, 1],
                           wspace=0.35,
                           hspace=0.35)
    ax00 = fig.add_subplot(gs[0, 0]); ax01 = fig.add_subplot(gs[0, 1], sharey=ax00); ax02 = fig.add_subplot(gs[0, 4], sharey=ax00)
    ax10 = fig.add_subplot(gs[1, 0], sharex=ax00, sharey=ax00); ax11 = fig.add_subplot(gs[1, 1], sharex=ax01, sharey=ax00); ax12 = fig.add_subplot(gs[1, 4], sharex=ax02, sharey=ax00)
    axes = np.array([[ax00, ax01, ax02], [ax10, ax11, ax12]])
    gs_temp_container = gs[:, 2].subgridspec(20, 1); cax_temp = fig.add_subplot(gs_temp_container[2:-2, 0])
    gs_rel_container = gs[:, 5].subgridspec(20, 1); cax_rel = fig.add_subplot(gs_rel_container[2:-2, 0]) # !! 修改 cax_diff !!

    plt.setp(ax01.get_yticklabels(), visible=False); plt.setp(ax02.get_yticklabels(), visible=False)
    plt.setp(ax11.get_yticklabels(), visible=False); plt.setp(ax12.get_yticklabels(), visible=False)
    plt.setp(ax00.get_xticklabels(), visible=False); plt.setp(ax01.get_xticklabels(), visible=False)
    plt.setp(ax02.get_xticklabels(), visible=False)

    sc_temp = None
    sc_rel = None
    
    if data_A['n']:
        sc_temp = ax00.scatter(data_A['n'], data_A['p'], c=data_A['T_sim_mK'], cmap=cmap_temp, vmin=vmin_temp, vmax=vmax_temp, s=25, edgecolors='k', lw=0.5)
        ax01.scatter(data_A['n'], data_A['p'], c=data_A['T_model_mK'], cmap=cmap_temp, vmin=vmin_temp, vmax=vmax_temp, s=25, edgecolors='k', lw=0.5)
        sc_rel = ax02.scatter(data_A['n'], data_A['p'], c=data_A['T_rel_err_percent'], cmap=cmap_rel, vmin=vmin_rel, vmax=vmax_rel, s=25, edgecolors='k', lw=0.5)

    if data_B['n']:
        if sc_temp is None:
            sc_temp = ax10.scatter(data_B['n'], data_B['p'], c=data_B['T_sim_mK'], cmap=cmap_temp, vmin=vmin_temp, vmax=vmax_temp, s=25, edgecolors='k', lw=0.5)
        else:
            ax10.scatter(data_B['n'], data_B['p'], c=data_B['T_sim_mK'], cmap=cmap_temp, vmin=vmin_temp, vmax=vmax_temp, s=25, edgecolors='k', lw=0.5)
        
        ax11.scatter(data_B['n'], data_B['p'], c=data_B['T_model_mK'], cmap=cmap_temp, vmin=vmin_temp, vmax=vmax_temp, s=25, edgecolors='k', lw=0.5)
        
        if sc_rel is None:
            sc_rel = ax12.scatter(data_B['n'], data_B['p'], c=data_B['T_rel_err_percent'], cmap=cmap_rel, vmin=vmin_rel, vmax=vmax_rel, s=25, edgecolors='k', lw=0.5)
        else:
            ax12.scatter(data_B['n'], data_B['p'], c=data_B['T_rel_err_percent'], cmap=cmap_rel, vmin=vmin_rel, vmax=vmax_rel, s=25, edgecolors='k', lw=0.5)

    ax00.set_title(r'TSAC: Simulation ($\mathcal{T}_{\mathrm{sim}}$)')
    ax01.set_title(r'TSAC: Model ($\mathcal{T}_{\mathrm{model}}$)')
    ax02.set_title(r'TSAC: Rel. Error $|\mathcal{T}_{\mathrm{sim}}\!-\!\mathcal{T}_{\mathrm{model}}|/\mathcal{T}_{\mathrm{model}}$') 
    
    ax10.set_title(r'DC: Simulation ($\mathcal{T}_{\mathrm{sim}}$)')
    ax11.set_title(r'DC: Model ($\mathcal{T}_{\mathrm{model}}$)')
    ax12.set_title(r'DC: Rel. Error $|\mathcal{T}_{\mathrm{sim}}\!-\!\mathcal{T}_{\mathrm{model}}|/\mathcal{T}_{\mathrm{model}}$') 
    
    ax00.set_ylabel('Noise Probability ($p$)'); ax10.set_ylabel('Noise Probability ($p$)') 
    ax10.set_xlabel('Number of Qubits ($n$)'); ax11.set_xlabel('Number of Qubits ($n$)'); ax12.set_xlabel('Number of Qubits ($n$)') 
    for ax in [ax10, ax11, ax12]: ax.set_xticks(simulated_n_values)
    for ax in axes.flat:
        ax.grid(True, linestyle=':', alpha=0.7); ax.set_yscale('log'); ax.minorticks_off()

    if sc_temp:
        fig.colorbar(sc_temp, cax=cax_temp).set_label(r'$\mathcal{T}$ (mK)', size=9) 
    if sc_rel:
        cb_rel = fig.colorbar(sc_rel, cax=cax_rel, format='%.1f%%')
        cb_rel.set_label('Relative Error (%)', size=9) # !! 修改标签 !!
        cb_rel.locator = plt.MaxNLocator(nbins=4) # 自动选择刻度
        cb_rel.update_ticks()

    fig.subplots_adjust(left=0.1, right=0.92, top=0.92, bottom=0.15)

    file_name = "comprehensive_comparison_aps_final_layout_appendix_v4_RelPercent.pdf" # 改名以区分
    full_path = os.path.join(output_dir, file_name);
    plt.savefig(full_path, dpi=300)
    print(f"Comprehensive figure (v4, Relative Percent, for appendix) saved to: {full_path}")
    plt.close(fig) # 关闭图形


if __name__ == "__main__":
    results_data = []
    output_dir = "D:/quantum_sim_figures" 

    all_n_to_simulate = sorted(list(SIMULATION_PARAMS.keys()))

    for n in all_n_to_simulate:
        p_list = SIMULATION_PARAMS[n]
        T_initial_mK_current = 163.0
        T_initial_K_current = T_initial_mK_current * 1e-3
        for p_cx in p_list:
            print(f"\n--- Simulating n={n}, p_cx={p_cx} ---")
            print("  Running Protocol A: TSAC...")
            T_sim_A, T_model_A = run_tsac_simulation(n, p_cx, TSAC_NUM_ROUNDS, T_initial_K_current)
            results_data.append({'protocol': 'TSAC', 'n': n, 'p': p_cx, 'T_sim': T_sim_A, 'T_model': T_model_A})
            print("  Running Protocol B: DC_Mirror...")
            try:
                T_sim_B, T_model_B = run_dc_mirror_simulation(n, p_cx, T_initial_K_current)
            except NameError: 
                T_sim_B, T_model_B = float('nan'), float('nan')
            results_data.append({'protocol': 'DC_Mirror', 'n': n, 'p': p_cx, 'T_sim': T_sim_B, 'T_model': T_model_B})

    print("\n--- Generating plots ---")
    plot_dc_mirror_only(results_data, output_dir, all_n_to_simulate)
    plot_comprehensive_comparison(results_data, output_dir, all_n_to_simulate)


--- Simulating n=3, p_cx=1e-05 ---
  Running Protocol A: TSAC...
  Running Protocol B: DC_Mirror...

--- Simulating n=3, p_cx=2e-05 ---
  Running Protocol A: TSAC...
  Running Protocol B: DC_Mirror...

--- Simulating n=3, p_cx=1e-06 ---
  Running Protocol A: TSAC...
  Running Protocol B: DC_Mirror...

--- Simulating n=3, p_cx=2e-07 ---
  Running Protocol A: TSAC...
  Running Protocol B: DC_Mirror...

--- Simulating n=4, p_cx=1e-06 ---
  Running Protocol A: TSAC...
  Running Protocol B: DC_Mirror...

--- Simulating n=4, p_cx=3e-06 ---
  Running Protocol A: TSAC...
  Running Protocol B: DC_Mirror...

--- Simulating n=4, p_cx=1e-07 ---
  Running Protocol A: TSAC...
  Running Protocol B: DC_Mirror...

--- Simulating n=4, p_cx=4e-08 ---
  Running Protocol A: TSAC...
  Running Protocol B: DC_Mirror...

--- Simulating n=5, p_cx=3e-07 ---
  Running Protocol A: TSAC...
  Running Protocol B: DC_Mirror...

--- Simulating n=5, p_cx=1e-08 ---
  Running Protocol A: TSAC...
  Running Protocol B: DC_